In [12]:
import numpy as np
import pandas as pd
import seaborn as sns # for visualiation
import statsmodels.formula.api as smf # linear modeling
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# Load data
data_features = pd.read_csv('./data/dengue_features_train.csv')
data_labels = pd.read_csv('./data/dengue_labels_train.csv')
data_test = pd.read_csv('./data/dengue_features_test.csv')

In [13]:
df = pd.DataFrame(columns=['Weeks', 'Error'])

In [18]:
for n in range(50, 107):
    
    ###################################################
    # Data Prep
    ###################################################
    
    #Features
    data_features_sj = data_features[data_features.city=='sj'].reset_index(drop = True)
    data_labels_sj = data_labels[data_labels.city=='sj'].reset_index(drop = True)
    data_sj = pd.merge(data_features_sj, data_labels_sj)
    data_sj = data_sj.drop(columns=['reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg'])

    data_sj = data_sj.fillna(method = 'ffill')
    data_sj['month'] = pd.to_datetime(data_sj['week_start_date']).dt.month
    data_sj['odd_year'] = data_sj.year.astype('int64') % 2 == 1
    data_sj['ndvi_mean'] = (data_sj['ndvi_ne'] + data_sj['ndvi_nw'] + data_sj['ndvi_se'] + data_sj['ndvi_sw']) / 4.0

    # data_sj = data_sj[data_sj['year'].isin(['1990','1991','1992','1993','1995','1996','1997','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008'])]

    data_sj['ndvi_mean_rolling_avg'] = data_sj['ndvi_mean'].rolling(window = n).mean()
    data_sj['ndvi_ne_rolling_avg'] = data_sj['ndvi_ne'].rolling(window = n).mean()
    data_sj['ndvi_nw_rolling_avg'] = data_sj['ndvi_nw'].rolling(window = n).mean()
    data_sj['ndvi_se_rolling_avg'] = data_sj['ndvi_se'].rolling(window = n).mean()
    data_sj['ndvi_sw_rolling_avg'] = data_sj['ndvi_sw'].rolling(window = n).mean()
    data_sj['precipitation_amt_mm_rolling_avg'] = data_sj['precipitation_amt_mm'].rolling(window = n).mean()
    data_sj['reanalysis_air_temp_k_rolling_avg'] = data_sj['reanalysis_air_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_avg_temp_k_rolling_avg'] = data_sj['reanalysis_avg_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_dew_point_temp_k_rolling_avg'] = data_sj['reanalysis_dew_point_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_max_air_temp_k_rolling_avg'] = data_sj['reanalysis_max_air_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_min_air_temp_k_rolling_avg'] = data_sj['reanalysis_min_air_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_precip_amt_kg_per_m2_rolling_avg'] = data_sj['reanalysis_precip_amt_kg_per_m2'].rolling(window = n).mean()
    data_sj['reanalysis_relative_humidity_percent_rolling_avg'] = data_sj['reanalysis_relative_humidity_percent'].rolling(window = n).mean()
    data_sj['reanalysis_tdtr_k_rolling_avg'] = data_sj['reanalysis_tdtr_k'].rolling(window = n).mean()
    data_sj['station_avg_temp_c_rolling_avg'] = data_sj['station_avg_temp_c'].rolling(window = n).mean()
    data_sj['station_diur_temp_rng_c_rolling_avg'] = data_sj['station_diur_temp_rng_c'].rolling(window = n).mean()
    data_sj['station_max_temp_c_rolling_avg'] = data_sj['station_max_temp_c'].rolling(window = n).mean()
    data_sj['station_min_temp_c_rolling_avg'] = data_sj['station_min_temp_c'].rolling(window = n).mean()
    data_sj['station_precip_mm_rolling_avg'] = data_sj['station_precip_mm'].rolling(window = n).mean()


    data_sj['week_start_date'] = pd.to_datetime(data_sj['week_start_date']).dt.month


    data_sj.drop(data_sj.head(n).index, inplace=True)

    # data_labels_sj_year = data_labels_sj.groupby(['year'])['total_cases'].stdev().to_frame(name = 'annual_cases').reset_index()
    # data_labels_sj = pd.merge(data_labels_sj, data_labels_sj_year)
    # data_labels_sj['pct_cases_year'] = data_labels_sj['total_cases'] / data_labels_sj['annual_cases']


    # Features - Normalize
    data_sj_n = MinMaxScaler().fit_transform(data_sj[data_sj.columns[4:43]])
    data_sj_n = pd.DataFrame(data_sj_n, columns = data_sj.columns[4:43], index=data_sj.index)
    data_sj_n = data_sj_n.drop(columns=['total_cases'])
    data_sj_n['month'] = data_sj['month']
    data_sj_n['odd_year'] = data_sj['odd_year']


    # Features Test
    data_sj_lastnweek = data_features[data_features.city=='sj'].reset_index(drop = True).tail(n)
    data_test_sj = data_test[data_test.city=='sj'].reset_index(drop = True)
    frames = [data_sj_lastnweek, data_test_sj]
    data_test_sj = pd.concat(frames).reset_index(drop = True)

    data_test_sj = data_test_sj.drop(columns=['reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg'])
    data_test_sj = data_test_sj.fillna(method = 'ffill')
    data_test_sj['month'] = pd.to_datetime(data_test_sj['week_start_date']).dt.month
    data_test_sj['odd_year'] = data_test_sj.year.astype('int64') % 2 == 1
    data_test_sj['ndvi_mean'] = (data_test_sj['ndvi_ne'] + data_test_sj['ndvi_nw'] + data_test_sj['ndvi_se'] + data_test_sj['ndvi_sw']) / 4.0

    data_test_sj['ndvi_mean_rolling_avg'] = data_test_sj['ndvi_mean'].rolling(window = n).mean()
    data_test_sj['ndvi_ne_rolling_avg'] = data_test_sj['ndvi_ne'].rolling(window = n).mean()
    data_test_sj['ndvi_nw_rolling_avg'] = data_test_sj['ndvi_nw'].rolling(window = n).mean()
    data_test_sj['ndvi_se_rolling_avg'] = data_test_sj['ndvi_se'].rolling(window = n).mean()
    data_test_sj['ndvi_sw_rolling_avg'] = data_test_sj['ndvi_sw'].rolling(window = n).mean()
    data_test_sj['precipitation_amt_mm_rolling_avg'] = data_test_sj['precipitation_amt_mm'].rolling(window = n).mean()
    data_test_sj['reanalysis_air_temp_k_rolling_avg'] = data_test_sj['reanalysis_air_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_avg_temp_k_rolling_avg'] = data_test_sj['reanalysis_avg_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_dew_point_temp_k_rolling_avg'] = data_test_sj['reanalysis_dew_point_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_max_air_temp_k_rolling_avg'] = data_test_sj['reanalysis_max_air_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_min_air_temp_k_rolling_avg'] = data_test_sj['reanalysis_min_air_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_precip_amt_kg_per_m2_rolling_avg'] = data_test_sj['reanalysis_precip_amt_kg_per_m2'].rolling(window = n).mean()
    data_test_sj['reanalysis_relative_humidity_percent_rolling_avg'] = data_test_sj['reanalysis_relative_humidity_percent'].rolling(window = n).mean()
    data_test_sj['reanalysis_tdtr_k_rolling_avg'] = data_test_sj['reanalysis_tdtr_k'].rolling(window = n).mean()
    data_test_sj['station_avg_temp_c_rolling_avg'] = data_test_sj['station_avg_temp_c'].rolling(window = n).mean()
    data_test_sj['station_diur_temp_rng_c_rolling_avg'] = data_test_sj['station_diur_temp_rng_c'].rolling(window = n).mean()
    data_test_sj['station_max_temp_c_rolling_avg'] = data_test_sj['station_max_temp_c'].rolling(window = n).mean()
    data_test_sj['station_min_temp_c_rolling_avg'] = data_test_sj['station_min_temp_c'].rolling(window = n).mean()
    data_test_sj['station_precip_mm_rolling_avg'] = data_test_sj['station_precip_mm'].rolling(window = n).mean()

    data_test_sj.drop(data_test_sj.head(n).index, inplace=True)

    # Features Test - Normalized
    data_test_sj_n = MinMaxScaler().fit_transform(data_test_sj[data_test_sj.columns[4:43]])
    data_test_sj_n = pd.DataFrame(data_test_sj_n, columns = data_test_sj.columns[4:43], index=data_test_sj.index)
    data_test_sj_n['month'] = data_test_sj['month']
    data_test_sj_n['odd_year'] = data_test_sj['odd_year']
    
    
    ###############################################################
    # Training Data
    ###############################################################

    train_features_sj, test_features_sj, train_outcome_sj, test_outcome_sj = train_test_split(
        data_sj_n[['month', 'odd_year', 'ndvi_sw_rolling_avg',
           'precipitation_amt_mm_rolling_avg',
           'reanalysis_dew_point_temp_k_rolling_avg',
           'reanalysis_precip_amt_kg_per_m2_rolling_avg',
           'reanalysis_relative_humidity_percent_rolling_avg',
           'station_diur_temp_rng_c_rolling_avg',
           'station_max_temp_c_rolling_avg']],
        data_sj['total_cases'],
        test_size = 0.3
    )
    
    ###############################################################
    # Grid Search
    ###############################################################
    
    params = {'n_neighbors':range(1, 50), 'weights':['uniform', 'distance']}
    folds = KFold(n_splits = 10, shuffle=True)
    grid_search = GridSearchCV(KNeighborsRegressor(), param_grid=params, cv=folds, scoring='neg_mean_absolute_error')
    grid_search.fit(train_features_sj, train_outcome_sj)
    
    grid_search.score(test_features_sj, test_outcome_sj)
    grid_search.cv_results_['params'][grid_search.best_index_]
    
    knr_reg = KNeighborsRegressor(n_neighbors = 5, weights = 'distance')
    knr_preds_sj = knr_reg.fit(train_features_sj, train_outcome_sj).predict(test_features_sj)
    
    print("done")
    #### ###################################
    #Adding to the DataFrame
    
    error = mean_absolute_error(test_outcome_sj, knr_preds_sj)
    print(n)
    print(error)

done
50
9.498185748347066
done
51
9.468452093826436
done
52
9.29205523665022
done
53
9.640606939160065
done
54
8.484041119052618
done
55
6.925783097814628
done
56
9.36753298249203
done
57
10.43587710254414
done
58
9.014871278120136
done
59
9.435025779447113
done
60
8.950134992425312
done
61
13.737326126601454
done
62
8.487949204997323
done
63
7.828216971479517
done
64
9.419990116945476
done
65
9.272075301203236
done
66
10.594827310925965
done
67
10.190673153593101
done
68
9.955388001448028
done
69
9.09514707180634
done
70
9.774276101619304
done
71
13.849272431320388
done
72
9.021329616254377
done
73
7.984138809103318
done
74
9.336710611995619
done
75
8.962827241214638
done
76
7.063686328753594
done
77
10.979078768239056
done
78
8.724522097337738
done
79
8.95959601900836
done
80
8.900082450248208
done
81
9.225679574523424
done
82
7.747197580212993
done
83
9.11929731918051
done
84
9.401224852064878
done
85
11.263287559506608
done
86
9.834908205401172
done
87
13.99395278971703
done
88
8.7

In [23]:
for n in range(35, 50):
    
    ###################################################
    # Data Prep
    ###################################################
    
    #Features
    data_features_sj = data_features[data_features.city=='sj'].reset_index(drop = True)
    data_labels_sj = data_labels[data_labels.city=='sj'].reset_index(drop = True)
    data_sj = pd.merge(data_features_sj, data_labels_sj)
    data_sj = data_sj.drop(columns=['reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg'])

    data_sj = data_sj.fillna(method = 'ffill')
    data_sj['month'] = pd.to_datetime(data_sj['week_start_date']).dt.month
    data_sj['odd_year'] = data_sj.year.astype('int64') % 2 == 1
    data_sj['ndvi_mean'] = (data_sj['ndvi_ne'] + data_sj['ndvi_nw'] + data_sj['ndvi_se'] + data_sj['ndvi_sw']) / 4.0

    # data_sj = data_sj[data_sj['year'].isin(['1990','1991','1992','1993','1995','1996','1997','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008'])]

    data_sj['ndvi_mean_rolling_avg'] = data_sj['ndvi_mean'].rolling(window = n).mean()
    data_sj['ndvi_ne_rolling_avg'] = data_sj['ndvi_ne'].rolling(window = n).mean()
    data_sj['ndvi_nw_rolling_avg'] = data_sj['ndvi_nw'].rolling(window = n).mean()
    data_sj['ndvi_se_rolling_avg'] = data_sj['ndvi_se'].rolling(window = n).mean()
    data_sj['ndvi_sw_rolling_avg'] = data_sj['ndvi_sw'].rolling(window = n).mean()
    data_sj['precipitation_amt_mm_rolling_avg'] = data_sj['precipitation_amt_mm'].rolling(window = n).mean()
    data_sj['reanalysis_air_temp_k_rolling_avg'] = data_sj['reanalysis_air_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_avg_temp_k_rolling_avg'] = data_sj['reanalysis_avg_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_dew_point_temp_k_rolling_avg'] = data_sj['reanalysis_dew_point_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_max_air_temp_k_rolling_avg'] = data_sj['reanalysis_max_air_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_min_air_temp_k_rolling_avg'] = data_sj['reanalysis_min_air_temp_k'].rolling(window = n).mean()
    data_sj['reanalysis_precip_amt_kg_per_m2_rolling_avg'] = data_sj['reanalysis_precip_amt_kg_per_m2'].rolling(window = n).mean()
    data_sj['reanalysis_relative_humidity_percent_rolling_avg'] = data_sj['reanalysis_relative_humidity_percent'].rolling(window = n).mean()
    data_sj['reanalysis_tdtr_k_rolling_avg'] = data_sj['reanalysis_tdtr_k'].rolling(window = n).mean()
    data_sj['station_avg_temp_c_rolling_avg'] = data_sj['station_avg_temp_c'].rolling(window = n).mean()
    data_sj['station_diur_temp_rng_c_rolling_avg'] = data_sj['station_diur_temp_rng_c'].rolling(window = n).mean()
    data_sj['station_max_temp_c_rolling_avg'] = data_sj['station_max_temp_c'].rolling(window = n).mean()
    data_sj['station_min_temp_c_rolling_avg'] = data_sj['station_min_temp_c'].rolling(window = n).mean()
    data_sj['station_precip_mm_rolling_avg'] = data_sj['station_precip_mm'].rolling(window = n).mean()


    data_sj['week_start_date'] = pd.to_datetime(data_sj['week_start_date']).dt.month


    data_sj.drop(data_sj.head(n).index, inplace=True)

    # data_labels_sj_year = data_labels_sj.groupby(['year'])['total_cases'].stdev().to_frame(name = 'annual_cases').reset_index()
    # data_labels_sj = pd.merge(data_labels_sj, data_labels_sj_year)
    # data_labels_sj['pct_cases_year'] = data_labels_sj['total_cases'] / data_labels_sj['annual_cases']


    # Features - Normalize
    data_sj_n = MinMaxScaler().fit_transform(data_sj[data_sj.columns[4:43]])
    data_sj_n = pd.DataFrame(data_sj_n, columns = data_sj.columns[4:43], index=data_sj.index)
    data_sj_n = data_sj_n.drop(columns=['total_cases'])
    data_sj_n['month'] = data_sj['month']
    data_sj_n['odd_year'] = data_sj['odd_year']


    # Features Test
    data_sj_lastnweek = data_features[data_features.city=='sj'].reset_index(drop = True).tail(n)
    data_test_sj = data_test[data_test.city=='sj'].reset_index(drop = True)
    frames = [data_sj_lastnweek, data_test_sj]
    data_test_sj = pd.concat(frames).reset_index(drop = True)

    data_test_sj = data_test_sj.drop(columns=['reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg'])
    data_test_sj = data_test_sj.fillna(method = 'ffill')
    data_test_sj['month'] = pd.to_datetime(data_test_sj['week_start_date']).dt.month
    data_test_sj['odd_year'] = data_test_sj.year.astype('int64') % 2 == 1
    data_test_sj['ndvi_mean'] = (data_test_sj['ndvi_ne'] + data_test_sj['ndvi_nw'] + data_test_sj['ndvi_se'] + data_test_sj['ndvi_sw']) / 4.0

    data_test_sj['ndvi_mean_rolling_avg'] = data_test_sj['ndvi_mean'].rolling(window = n).mean()
    data_test_sj['ndvi_ne_rolling_avg'] = data_test_sj['ndvi_ne'].rolling(window = n).mean()
    data_test_sj['ndvi_nw_rolling_avg'] = data_test_sj['ndvi_nw'].rolling(window = n).mean()
    data_test_sj['ndvi_se_rolling_avg'] = data_test_sj['ndvi_se'].rolling(window = n).mean()
    data_test_sj['ndvi_sw_rolling_avg'] = data_test_sj['ndvi_sw'].rolling(window = n).mean()
    data_test_sj['precipitation_amt_mm_rolling_avg'] = data_test_sj['precipitation_amt_mm'].rolling(window = n).mean()
    data_test_sj['reanalysis_air_temp_k_rolling_avg'] = data_test_sj['reanalysis_air_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_avg_temp_k_rolling_avg'] = data_test_sj['reanalysis_avg_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_dew_point_temp_k_rolling_avg'] = data_test_sj['reanalysis_dew_point_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_max_air_temp_k_rolling_avg'] = data_test_sj['reanalysis_max_air_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_min_air_temp_k_rolling_avg'] = data_test_sj['reanalysis_min_air_temp_k'].rolling(window = n).mean()
    data_test_sj['reanalysis_precip_amt_kg_per_m2_rolling_avg'] = data_test_sj['reanalysis_precip_amt_kg_per_m2'].rolling(window = n).mean()
    data_test_sj['reanalysis_relative_humidity_percent_rolling_avg'] = data_test_sj['reanalysis_relative_humidity_percent'].rolling(window = n).mean()
    data_test_sj['reanalysis_tdtr_k_rolling_avg'] = data_test_sj['reanalysis_tdtr_k'].rolling(window = n).mean()
    data_test_sj['station_avg_temp_c_rolling_avg'] = data_test_sj['station_avg_temp_c'].rolling(window = n).mean()
    data_test_sj['station_diur_temp_rng_c_rolling_avg'] = data_test_sj['station_diur_temp_rng_c'].rolling(window = n).mean()
    data_test_sj['station_max_temp_c_rolling_avg'] = data_test_sj['station_max_temp_c'].rolling(window = n).mean()
    data_test_sj['station_min_temp_c_rolling_avg'] = data_test_sj['station_min_temp_c'].rolling(window = n).mean()
    data_test_sj['station_precip_mm_rolling_avg'] = data_test_sj['station_precip_mm'].rolling(window = n).mean()

    data_test_sj.drop(data_test_sj.head(n).index, inplace=True)

    # Features Test - Normalized
    data_test_sj_n = MinMaxScaler().fit_transform(data_test_sj[data_test_sj.columns[4:43]])
    data_test_sj_n = pd.DataFrame(data_test_sj_n, columns = data_test_sj.columns[4:43], index=data_test_sj.index)
    data_test_sj_n['month'] = data_test_sj['month']
    data_test_sj_n['odd_year'] = data_test_sj['odd_year']
    
    
    ###############################################################
    # Training Data
    ###############################################################

    train_features_sj, test_features_sj, train_outcome_sj, test_outcome_sj = train_test_split(
        data_sj_n[['month', 'odd_year', 'ndvi_sw_rolling_avg',
           'precipitation_amt_mm_rolling_avg',
           'reanalysis_dew_point_temp_k_rolling_avg',
           'reanalysis_precip_amt_kg_per_m2_rolling_avg',
           'reanalysis_relative_humidity_percent_rolling_avg',
           'station_diur_temp_rng_c_rolling_avg',
           'station_max_temp_c_rolling_avg']],
        data_sj['total_cases'],
        test_size = 0.3
    )
    
    ###############################################################
    # Grid Search
    ###############################################################
    
    params = {'n_neighbors':range(1, 50), 'weights':['uniform', 'distance']}
    folds = KFold(n_splits = 10, shuffle=True)
    grid_search = GridSearchCV(KNeighborsRegressor(), param_grid=params, cv=folds, scoring='neg_mean_absolute_error')
    grid_search.fit(train_features_sj, train_outcome_sj)
    
    grid_search.score(test_features_sj, test_outcome_sj)
    grid_search.cv_results_['params'][grid_search.best_index_]
    
    knr_reg = KNeighborsRegressor(n_neighbors = 5, weights = 'distance')
    knr_preds_sj = knr_reg.fit(train_features_sj, train_outcome_sj).predict(test_features_sj)
    
    print("done")
    #### ###################################
    #Adding to the DataFrame
    
    error = mean_absolute_error(test_outcome_sj, knr_preds_sj)
    print(n)
    print(error)

done
35
10.955433610004155
done
36
9.831874813184081
done
37
8.513022305632836
done
38
13.680295267033914
done
39
8.17995400411768
done
40
8.942788230215958
done
41
8.730190371677658
done
42
8.898897591844284
done
43
9.000084173620403
done
44
8.573605703381702
done
45
7.403373526301191
done
46
8.95538740234929
done
47
10.128934586906938
done
48
9.44724598260084
done
49
9.520445363631994
